# Politics in Truman

In [1]:
SCHOOL="truman"
SUBJECT="editorial"
START_YEAR=2015
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(399, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,Trading Card Games club welcomes all skill levels,https://tmn.truman.edu/blog/editorial/trading-...,Trading Card Games club welcomes all skill lev...,2023,3,8
1,Our View: Preserve the Kirk Building,https://tmn.truman.edu/blog/editorial/our-view...,Our View: Preserve the Kirk Building\nWorkers ...,2023,3,2


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

4.625585


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015], dtype=int64)

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2015, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2015 ... 51
2015	51	106.353055	2.085354019607843
Year: 2016 ... 92
2016	92	-34.74335500000001	-0.37764516304347834
Year: 2017 ... 45
2017	45	1.1861910000000027	0.026359800000000058
Year: 2018 ... 46
2018	46	44.470469	0.9667493260869565
Year: 2019 ... 65
2019	65	137.776204	2.119633907692308
Year: 2020 ... 48
2020	48	33.21214799999999	0.6919197499999998
Year: 2021 ... 28
2021	28	14.439305000000003	0.5156894642857144
Year: 2022 ... 20
2022	20	-1.2685999999999975	-0.06342999999999988


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)